# 📊 Predição de séries temporais com SARIMA

## 📊 Contexto do Projeto

📁 **Dataset utilizado:**  
O conjunto de dados analisado é o clássico **Air Passengers Dataset**, que contém o número mensal de passageiros de uma companhia aérea internacional ao longo do tempo.

🗓️ **Período de observação:**  
De **janeiro de 1949** até **dezembro de 1960**, totalizando **144 registros mensais**.

---

### 🔍 Por que esse dataset é interessante?

Este dataset é frequentemente utilizado para estudos e modelagens de **séries temporais**, devido às suas características bem definidas:

- 📈 **Tendência:**  
  O número de passageiros cresce ao longo dos anos, indicando uma **tendência de alta**.

- 📆 **Sazonalidade:**  
  A série exibe um **padrão sazonal anual**, ou seja, o comportamento se repete aproximadamente a cada **12 meses**.

- 🌪️ **Flutuações aleatórias:**  
  Existem pequenas **variações** além da tendência e sazonalidade, que podem estar relacionadas a **ruídos** ou **eventos específicos**.

---

### 🧠 Estratégia de Modelagem

📌 Para este estudo, foi escolhido o modelo **SARIMA** (*Seasonal AutoRegressive Integrated Moving Average*), pois:

> ⚠️ A série possui uma **sazonalidade explícita** — o que torna o SARIMA uma opção ideal para capturar tanto a tendência quanto os ciclos sazonais.


## Imports

In [ ]:
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX

## Pipeline 

### Carregamento dos dados 

In [ ]:
# Carregar o dataset AirPassengers
data = sm.datasets.get_rdataset('AirPassengers', package='datasets')
df = data.data

#Renomeando colunas
df = df.rename(columns={'time':'Month', 'value':'Passengers'})

#Convertendo valores decimais em data
def decimal_to_date(decimal):
    year = int(decimal)
    # Extrai fração e converte em mês
    month_fraction = decimal - year
    month = int(round(month_fraction * 12)) + 1
    if month > 12:
        month = 1
        year += 1
    return pd.Timestamp(year=year, month=month, day=1)

df['Month'] = df.Month.apply(decimal_to_date)

#Setando a coluna mês para index
df.reset_index()
df.set_index('Month', inplace=True)

### Exploração dos dados (EDA)

#### Visualização inicial dos dados

In [ ]:
df.head()

#### Resumo estatístico das variáveis numéricas

In [ ]:
df.describe()

#### Verificar a existência de valores nulos

Não existem valores nulos

In [ ]:
df.isnull().sum()

#### Visualizando a série temporal: Verificando a exitência de sazonalidade

Analisando visualmente o gráfico, vemos que existem padrões que se repetem o que nos remete a existência de sazonalidade

In [ ]:
df.Passengers.plot()
plt.ylabel('Passengers')
plt.show()

#### Vericar de quanto em quanto tempo se repetem os ciclos de sazonalidade

Analisando o gráfico ACF vemos que a sazonalidade acontece de 12 em 12 meses, ou seja, a cada ano

In [ ]:
plot_acf(df, lags=48)  # lags ajustados conforme o necessário
plt.show()

#### Decompondo a série temporal em componentes (Tendência, Sazonalidade e Resíduos)

In [ ]:
# Decomposição com o período de 12 meses
result = seasonal_decompose(df, model='additive', period=12)
result.plot()
plt.show()

#### Aplicando a diferenciação sazonal

Como a série é sazona em 12, aplicamos a diferenciação sazonal para remover a sazonalidade da séria e deixá-la estacionária para facilitar modelagem e escolha de parẽmetros do modelo

In [ ]:
# Diferenciação sazonal de ordem 1 (D=1)
df_seasonal_diff = df.diff(12).dropna()

# Plot da série diferenciada sazonalmente
plt.plot(df_seasonal_diff, label="Seasonally Differenced Data (D=1)")
plt.title("Diferenciação sazonal (Sazonalidade de 12 meses)")
plt.xlabel("Ano")
plt.ylabel("Contagem de Passageiros Diferenciada Sazonalmente")
plt.legend()
plt.show()

#### Realizando o teste de estacionaridade ADF

Realizando o teste de estacionaridade, vemos que com a diferenciação sazonal, a séria passou no teste de estacionaridade

In [ ]:
def testar_adf(serie_temporal):
    """
    Realiza o teste ADF (Augmented Dickey-Fuller) para verificar se a série temporal é estacionária.
    
    Parâmetros:
    serie_temporal (pandas.Series): A série temporal a ser testada.
    
    Retorna:
    str: Mensagem com o resultado do teste.
    """
    
    # Realiza o Teste ADF
    resultado = adfuller(serie_temporal)
    
    # Estatísticas do Teste
    estatistica_adf = resultado[0]
    valor_p = resultado[1]
    lags_usados = resultado[2]
    num_observacoes = resultado[3]
    valores_criticos = resultado[4]
    
    # Exibe os resultados
    print('Estatística ADF:', estatistica_adf)
    print('Valor-p:', valor_p)
    print('Número de lags usados:', lags_usados)
    print('Número de observações:', num_observacoes)
    print('Valores críticos:')
    for chave, valor in valores_criticos.items():
        print(f'\t{chave}: {valor}')
    
    # Interpretação do resultado
    if valor_p < 0.05:
        return "Rejeita a hipótese nula: A série é estacionária."
    else:
        return "Não rejeita a hipótese nula: A série não é estacionária."

testar_adf(df_seasonal_diff)

### Teste de Modelos: SARIMA

#### Definição dos parâmetros do modelo: p, q, m e d

- Visualizando os gráficos ACF e PACF vamos definir os parâmetros do modelo SARIMA:

- O valor de d é igual a 1 pq foi aplicada a diferenciação sazonal de primeira ordem.

- O valor de m que seria a frequência sazonal serie de 12 mêses definido na análise explorarória acima.

- Analizando o AFC vemos que o lado posterior acaba sendo menor que o anterior e que o valor de q pode ser muito menor (q = 0, q = 1)

- Analisando o PACF vemos que a partir do segundo lag, vemos uma queda brusca isso remete que o valor de p é muito pequeno (p = 1 ou p = 2).

In [ ]:
# Gerar gráfico de ACF
plt.figure(figsize=(12, 6))
plot_acf(df, lags=40)  # Ajuste o número de lags conforme necessário
plt.title("ACF - Autocorrelação")
plt.show()

# Gerar gráfico de PACF
plt.figure(figsize=(12, 6))
plot_pacf(df, lags=40)  # Ajuste o número de lags conforme necessário
plt.title("PACF - Autocorrelação Parcial")
plt.show()

#### Definição dos parâmetros do modelo: P Q e D

- No grafico PACF vemos o primeiro e segundo lag com picos significativos ou seja ou valor o valor de P pode assumir (P = 1 ou P 2).
- O valor de D é a ordem de diferenciação, como a diferenciação sazonal é de ordem 1 então (D = 1).
- No gráfico ACF os lags sazonais começam em um pico alto, vão diminuindo ao longo do tempo e chegando a zero o que indica que o valor de Q pode assumir (Q = 1).

In [ ]:
# Gerar gráfico de ACF
plt.figure(figsize=(12, 6))
plot_acf(df_seasonal_diff, lags=40)  # Ajuste o número de lags conforme necessário
plt.title("ACF - Autocorrelação")
plt.show()

# Gerar gráfico de PACF
plt.figure(figsize=(12, 6))
plot_pacf(df_seasonal_diff, lags=40)  # Ajuste o número de lags conforme necessário
plt.title("PACF - Autocorrelação Parcial")
plt.show()

#### Treinamento do modelo SARIMA

In [ ]:
# Treinamento do SARIMA

df.index.freq = 'MS'
model = SARIMAX(df['Passengers'], order=(1,1,1), seasonal_order=(1,1,1,12))

In [ ]:
result = model.fit()

# Previsão de 12 meses a frente
forecast = result.forecast(steps=12)

# Plot actual vs forecasted values
plt.figure(figsize=(12, 6))
plt.plot(df, label="Atual")
plt.plot(forecast, label="Previsto", linestyle="dashed", color='red')
plt.title("SARIMA Forecast")
plt.xlabel("Ano")
plt.ylabel("Passengers")
plt.legend()
plt.show()

## Resumo dos Resultados do Forecast SARIMA
Foi ajustado um modelo SARIMA(1,1,1)(1,1,1,12) para a série temporal de passageiros aéreos mensais, contemplando tanto a diferenciação regular quanto a sazonal, com sazonalidade anual (12 meses).

Durante o processo de otimização via L-BFGS-B, o modelo apresentou boa convergência:

O valor da função objetivo (Log-Likelihood) foi minimizado de 3.5284 para 3.5149.
O algoritmo convergiu após 23 iterações e 27 avaliações de função, com um gradiente projetado final de 3.24e-05, indicando um ajuste estável e eficiente.
A mensagem de convergência final: "REL_REDUCTION_OF_F <= FACTR * EPSMCH", confirma que o critério de parada baseado na redução relativa da função foi atendido com sucesso.
Com isso, o modelo foi considerado bem ajustado e foi utilizado para realizar um forecast de 12 meses à frente da série original, proporcionando previsões com base nos padrões de tendência e sazonalidade detectados.